<a href="https://colab.research.google.com/github/shivanisingh28/sentimentanalysis/blob/master/kerasAssgmnt2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')
import os

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
# Import the pandas library to read our dataset 
import pandas as pd
# Get the train/test split package from sklearn for preparing our dataset to # train and test the model with 
from sklearn.model_selection import train_test_split
# Import the numpy library to work with and manipulate the data
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
sns.set(color_codes=True)

import nltk 
import random 
from nltk.tokenize import word_tokenize
nltk.download('punkt') 
nltk.download('stopwords')
nltk.download('wordnet') 

Instructions for updating:
non-resource variables are not supported in the long term
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [3]:
import csv
import urllib.request as urllib2
from nltk import FreqDist
# Importing data using url
url = 'https://raw.githubusercontent.com/cacoderquan/Sentiment-Analysis-on-the-Rotten-Tomatoes-movie-review-dataset/master/train.tsv'

# reading data using pandas and converting into dataframe
df = pd.read_csv(urllib2.urlopen(url),delimiter='\t',encoding='utf-8')
df.head()


,PhraseId,SentenceId,Phrase,Sentiment
0,1,1,A series of escapades demonstrating the adage ...,1
1,2,1,A series of escapades demonstrating the adage ...,2
2,3,1,A series,2
3,4,1,A,2
4,5,1,series,2


In [4]:
df = df.sample(frac=1).reset_index(drop=True)
df.head()
# Check class imbalance in tokenized sentences
df['Sentiment'].value_counts()

2    79582
3    32927
1    27273
4     9206
0     7072
Name: Sentiment, dtype: int64

In [0]:
# Get number of unique sentences
numSentences = df['SentenceId'].max()

In [0]:
# extract full sentences only from the dataset
fullSentences = []
curSentence = 0
for i in range(df.shape[0]):
  if df['SentenceId'][i]> curSentence:
    fullSentences.append((df['Phrase'][i], df['Sentiment'][i]))
    curSentence = curSentence +1

In [0]:
# put data into a df
fullSentDf = pd.DataFrame(fullSentences,
                                columns=['Phrase', 'Sentiment'])

In [8]:
fullSentDf['Sentiment'].value_counts()

2    4232
3    1769
1    1591
4     519
0     433
Name: Sentiment, dtype: int64

In [0]:
documents=[]
X_data = np.array(fullSentDf['Phrase'].values.tolist())
Y_data = np.array(fullSentDf['Sentiment'].values.tolist())

for i in range(fullSentDf.shape[0]):
  tmpWords = word_tokenize(fullSentDf['Phrase'][i])
  documents.append((tmpWords, fullSentDf['Sentiment'][i]))


In [0]:
from nltk.corpus import stopwords 
from nltk.stem import WordNetLemmatizer, PorterStemmer, LancasterStemmer 
porter = PorterStemmer() 
lancaster=LancasterStemmer() 
wordnet_lemmatizer = WordNetLemmatizer() 
stopwords_en = stopwords.words("english") 
punctuations="?:!.,;'\"-()"
#parameters to adjust to see the impact on outcome 
remove_stopwords = True
useStemming = False
useLemma = False
removePuncs = True


In [0]:
for l in range(len(documents)):                   #For each review document 
  label = documents[l][1]                         #Save review label 
  tmpReview = []                                  #Placeholder list for new review 
  for w in documents[l][0]:                       #For each word this is review 
    newWord = w                                   #Set newWork to be the updated word 
    if remove_stopwords and (w in stopwords_en):  #if the word is a stopword & we want to remove stopwords 
      continue                                    #skip the word and don’t had it to the normalized review 
    if removePuncs and (w in punctuations):       #if the word is a punc. & we want to remove punctuations 
      continue                                    #skip the word and don’t had it to the normalized review 
    if useStemming:
      #if useStemming is set to True 
      #Keep one stemmer commented out 
      #newWord = porter.stem(newWord) #User porter stemmer 
      newWord = lancaster.stem(newWord) #Use Lancaster stemmer 
    if useLemma: 
      newWord = wordnet_lemmatizer.lemmatize(newWord) 
    tmpReview.append(newWord)                     #Add normalized word to the tmp review 
  documents[l] = (tmpReview, label)             #Update the reviews list with clean review 
  documents[l] = (' '.join(tmpReview), label) 


In [12]:
df = pd.DataFrame(documents, columns=['text', 'sentiment']) 
df.head()

,text,sentiment
0,cartoon,2
1,really goes downhill,0
2,pushes,2
3,deeply unsettling experience,1
4,It 's vampires damned Queen Damned -- viewers ...,0


In [0]:
X_train, X_test, Y_train, Y_test = train_test_split(df['text'],  df['sentiment'], test_size=0.3, random_state=2003)

In [14]:
from sklearn.feature_extraction.text import CountVectorizer , TfidfVectorizer
from keras.utils import to_categorical

vectorizer = TfidfVectorizer(max_features = 2000,ngram_range=(2, 2)) 
X = vectorizer.fit_transform(df["text"]) 
Y = df['sentiment'] 
 
X_train = vectorizer.transform(X_train).toarray()
Y_train = Y_train 
X_test = vectorizer.transform(X_test).toarray()
Y_test = Y_test

Using TensorFlow backend.


In [0]:
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv1D, MaxPooling1D
from keras import backend as K

In [0]:
def recallfunction(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

In [0]:
def precisionfunction(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

In [0]:
def f1function(y_true, y_pred):
    precision = precisionfunction(y_true, y_pred)
    recall = recallfunction(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [0]:
batch_size = 64
num_classes = 5
epochs = 12

In [21]:
X_train.shape
# Y_train = to_categorical(Y_train,5)
# y_test = to_categorical(y_test,5)

(5980, 2000)

In [0]:
Y_train = keras.utils.to_categorical(Y_train, num_classes)
Y_test = keras.utils.to_categorical(Y_test, num_classes)


In [23]:

model = Sequential()
model.add(Conv1D(filters=64, kernel_size=3,
                 activation='relu',
                 input_shape=(2000,1)))
model.add(Conv1D(128, kernel_size=5, activation='relu'))
model.add(MaxPooling1D(pool_size=1))
model.add(Dropout(rate = 0.25))
model.add(Flatten())
model.add(Dense(num_classes, activation='softmax'))







Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [26]:
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adam(),
              metrics=['accuracy',f1function,precisionfunction, recallfunction])

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_1 (Conv1D)            (None, 1998, 64)          256       
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 1994, 128)         41088     
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 1994, 128)         0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 1994, 128)         0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 255232)            0         
_________________________________________________________________
dense_1 (Dense)              (None, 5)                 1276165   
Total params: 1,317,509
Trainable params: 1,317,509
Non-trainable params: 0
____________________________________________

In [0]:
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)

In [37]:
model.fit(X_train, Y_train,
          batch_size=64,
          epochs=8)
model.save('/content/drive/My Drive/my_model.h5')
score = model.evaluate(X_test, Y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])
print('Test F1 score:', score[2])
print('Test precision:', score[3])
print('Test recall:', score[4])

Epoch 1/8
5980/5980 [==============================] - 2s 294us/step - loss: 0.9735 - acc: 0.6005 - f1function: 0.5923 - precisionfunction: 0.6170 - recallfunction: 0.5749
Epoch 2/8
5980/5980 [==============================] - 2s 291us/step - loss: 0.9736 - acc: 0.6025 - f1function: 0.5742 - precisionfunction: 0.6280 - recallfunction: 0.5505
Epoch 3/8
5980/5980 [==============================] - 2s 292us/step - loss: 0.9730 - acc: 0.6007 - f1function: 0.5986 - precisionfunction: 0.6121 - recallfunction: 0.5870
Epoch 4/8
5980/5980 [==============================] - 2s 292us/step - loss: 0.9707 - acc: 0.6065 - f1function: 0.5877 - precisionfunction: 0.6272 - recallfunction: 0.5669
Epoch 5/8
5980/5980 [==============================] - 2s 290us/step - loss: 0.9751 - acc: 0.6050 - f1function: 0.5578 - precisionfunction: 0.6505 - recallfunction: 0.5227
Epoch 6/8
5980/5980 [==============================] - 2s 286us/step - loss: 0.9721 - acc: 0.6047 - f1function: 0.5436 - precisionfunction: 

In [0]:
from keras.models import load_model

model = load_model('/content/drive/My Drive/my_model.h5',custom_objects={'f1function':f1function,'precisionfunction':precisionfunction,'recallfunction':recallfunction})
model.save('my_model.h5')  # creates a HDF5 file
#del model  # deletes the existing model